## éclat — A PhD‑level Guidebook (System Overview)

### Executive summary

This guidebook presents éclat, an Agentic AI orchestration system for consumer lending designed to recover customer trust and lift conversion by treating the loan application as a temporally extended conversational event rather than a stateless form fill. At its core is a Master Agent (an observable, policy‑driven orchestrator) that reasons over an interaction stream, maintains a temporal state graph of the customer's decision readiness, and coordinates specialized Worker Agents for Sales, Verification, Underwriting, and Sanctioning. The design emphasizes four pillars: reproducibility, interpretability, auditability, and safety—each operationalized with deterministic transforms, linear surrogates for attribution, append‑only artifacts with cryptographic provenance, and conservative gating for high‑risk actions.

### How to use this document

Treat this notebook as a comprehensive design and runbook. Sections combine rigorous theoretical foundations (computer science, machine learning, natural language processing, and finance) with practical contracts and operational checklists. Use the Appendix for artifact schemas, templates, and recommended evaluation scripts. This document is intentionally self‑contained and citation‑aware; follow the references in the bibliography when validating assumptions.

### 1. Formal problem statement

Let $X_{e t}$ be the chronological interaction stream for a prospective borrower: timestamps, raw tokens, pause durations, edit events, and UI signals. Let $M$ be persistent metadata (lead attributes, session context) and $E$ be external service signals (credit bureau, KYC). The system must produce, at decision times $t$, a triplet: an orchestration directive $athcal{D}_t$ for worker invocation, a safe, templated utterance $T_t$, and an auditable artifact $Z_t$. Formally, design a deterministic mapping $F$ so that for all input trajectories encountered in production the outputs remain explainable and reproducible:

$$ (athcal{D}_t, T_t, Z_t) = F(X_{e t}, M, E; heta, i) $$

where $heta$ denotes auditable numeric parameters (weights, thresholds) and $i$ denotes policy artifacts (persona templates, guarded escalation rules). The design objective is to maximize conversion subject to safety constraints (privacy, fairness, regulatory compliance).

### 2. Theoretical foundations — concise but rigorous

2.1 Temporal state graphs (CS & dynamical systems)

Model the conversation as a time‑indexed directed graph $G_t=(V_t,E_t)$ where nodes represent semantic conversational states and weighted edges capture transition propensities informed by observed signals. Edge weights update as an exponential moving sum of signals:

$$ w_{ij}(t) = um_{	aue t} lpha^{t-	au} s_{ij}(	au), uad 0<lpha<1, $$

This yields a Markovian approximation of short‑term dynamics and supports reachability and expected hitting‑time analysis under conservative approximations—useful for formal guarantees about escalation probabilities and human handoff latency bounds.

2.2 Linear surrogates, attribution, and interpretability (ML)

Use linear surrogates $S = w^	op f$ where $f=hi(X)$ are deterministic features. The per‑feature contribution is exactly $w_i f_i$, enabling precise explanations and simple constrained optimization for fairness adjustments. When probability calibration is needed, apply a monotone link $igma$: $p = igma(S)$ with $igma$ the logistic function. For small feature sets, use constrained linear regression with L2 penalty to estimate $w$ from labeled data while preserving stability.

2.3 Attribution & fairness (SHAP conceptual link)

SHAP values approximate per‑feature contributions for complex models; with a linear surrogate they reduce to exact contributions. Define persuasion‑attribution vector $a$ (subset of features associated with persuasive or urgency cues). A manipulation index is computed as a weighted sum: $M = a^	op eta$; policies map $M$ to mitigation actions (e.g., reduce urgency by $ambda M$). This analytic approach permits formal policy tuning and backtesting.

2.4 Financial decision rules (underwriting logic)

Underwriting is expressed as symbolic constraints plus numeric thresholds. For example, given requested loan amount $L$, pre‑approved limit $P$, credit score $C$, and salary $Y$, a simple decision rule set is:

- Approve if $L e P$.
- If $P < L e 2P$, require income proof and approve only if $EMI(L,T)/Y e 0.5$ where $T$ is tenure and $EMI(dot)$ is the standard annuity formula.
- Reject if $L>2P$ or $C<700$.

This rule set is intentionally declarative and auditable; it maps directly to the canonical artifact fields recorded for each decision.

### 3. System components and precise contracts

3.1 Ingest (contract): validate raw events, attach `trace_id`, normalize timestamps, and return `event_summary`. Minimal contract fields: `{trace_id, lead_id, timestamp, event_type, raw_text, meta}`.

3.2 Feature Extractor (contract): produce `features` dictionary from `event_summary` using deterministic rules. Document `phi` precisely: token cues, pause thresholds, edit rates, sentiment buckets, and derived fields (e.g., `urgency_token_count`).

3.3 Master Agent (contract): API primitives — `decide(state_snapshot) -> {action, payload, reason}`. Inputs: `state_snapshot` (graph summary, recent features, external signals). Outputs: action (`invoke`, `send_template`, `escalate`), payload (typed JSON), and `reason` (human readable). Always emit artifact `Z` containing the snapshot and the chosen action.

3.4 Worker Agents (contracts): each agent accepts a typed payload and returns a typed result. Examples: Sales returns `{utterance_suggestion, offer_object, persuasion_attributions}`; Verification returns `{verified, confidence, corrections}`; Underwriting returns `{score, decision, shap_attributions, explanation_template}`; Sanction returns `{doc_uri, template_commit, content_hash}`.

### 4. Master Agent internals — algorithms and policy primitives

4.1 Readiness and risk gating

Compute Readiness $R_t$ as a monotonic function of positive signals (planning tokens, timeline markers, affirmative edits) and compute Risk $Q_t$ from credit proxies and anomaly detectors. The gating policy is a conservative mapping: if $Q_t>q_0$ or model uncertainty $U_t>u_0$ then `escalate`; else select worker with highest expected utility under safety constraints. Expected utility can be approximated with a linear surrogate over features of `state_snapshot` calibrated offline.

4.2 Temporal graph updates and interventions

On each event, update edge weights via exponential decay and add a signal increment. Use these weights to compute features such as `abandon_pressure = w_{current,abandon}` and `hesitation_index = um_{in H} w_{current,i}`. Define intervention triggers as threshold crossings on these indices, producing deterministic, testable behaviors (e.g., on `hesitation_index>h_0` send a reframing template).

4.3 Artifact provenance

Every emitted artifact `Z` includes `{trace_id, timestamp, state_snapshot, features, weights, decision, policy_version, generator_version, signature}`. Sign artifacts with HMAC using a deployment key and store generator_version as a semantic tag linking to code and policy commits for reproducibility.

### 5. Fairness and safety — formal mitigation

Define the manipulation score $M$ over persuasion attributions and policy weights. Mitigation policy: if $M>M_{max}$ and credit proxy $C<C_{min}$ then reduce the Sales Agent’s urgency contribution by $elta U = ambda (M-M_{max})$ and log a `forced_reframe` event. This rule is mathematically simple, auditable, and admits offline backtesting by simulating counterfactuals on historical interaction traces.

### 6. Market analysis and business rationale

6.1 Opportunity sizing — illustrative model

Let baseline conversion rate be $p_0$ and daily unique leads be $N$. A reduction in abandonment by $elta$ (absolute) increases converted leads by $Nelta$, producing incremental revenue $elta R = Nelta dot ARPU$. For NBFCs with high acquisition cost, even modest reductions in abandonment yield high ROI. Empirical BFSI benchmarks place abandonment at 30–45% at friction points; targeting a conservative 5% absolute improvement is a realistic product goal with measurable commercial upside.

6.2 Competitive differentiation

éclat’s principal differentiator is auditable orchestration coupled with psychologically informed pacing. This combination enables both higher conversion and regulatory defensibility compared to opaque LLM assistants or brittle scripted chatbots.

### 7. Evaluation protocol & KPIs

Define A/B experiments with shadow mode where Master Agent decisions are logged but not executed. Primary KPIs: application completion, approval‑to‑disbursal conversion, NPS uplift, and reduction in manual escalations. Safety KPIs: manipulation alert rate, fraction of artifacts with full provenance, and mean time to human review for escalations.

### 8. Deployment, monitoring, and runbook

- Deploy stateless Master Agent instances with a shared TimescaleDB event store and a policy configuration service (feature flags, policy versioning).
- Start in shadow mode, enable Sales Agent suggestions only after 2–4 weeks of traffic validation, then enable automated actions behind policy gates.
- Monitor drift, escalation spikes, and generator_version mismatches; automate rollback when threshold breaches are detected.

### 9. Regulatory mapping and documentation

Map artifact fields to regulatory disclosure requirements: reasons for denial (SHAP attributions), timelines for decision, and records of human oversight. Maintain retained artifacts in encrypted JSONB with schema_version and signed digests for audits.

### 10. Limitations and future work

- Linear surrogates are intentionally conservative; future work may explore hybrid models with calibrated post‑hoc explanations.
- Psychological state estimation is noisy; add human labeling for improved calibration and bias auditing.
- Expand to multimodal signals (voice, video) with explicit privacy controls.

### 11. Acceptance checklist

- Artifact schema defined and sample artifacts present.
- Persona templates recorded with commit hashes.
- Minimal end‑to‑end demo (Sales + Verification) validated in shadow mode.
- Monitoring alerts configured for drift and escalation spikes.

### Bibliography (select)

- Lundberg, S.M. & Lee, S.-I. (2017). A Unified Approach to Interpreting Model Predictions. NIPS. (SHAP).
- Clark, H. H., & Brennan, S. E. (1991). Grounding in communication. Perspectives on socially shared cognition.
- Doshi‑Velez, F. & Kim, B. (2017). Towards A Rigorous Science of Interpretable Machine Learning. arXiv.
- Ribeiro, M.T., Singh, S., & Guestrin, C. (2016). 'Why Should I Trust You?': Explaining the Predictions of Any Classifier. KDD.
- Kannan, A. et al. (2023). Operationalizing Responsible AI—A Practical Guide.